In [1]:
#This notebook aims to measure the SNR of each star we are looking at in K-band

import numpy as np
import glob
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, ListedColormap
from scipy.stats import sigmaclip, binned_statistic as bs
from scipy.spatial import KDTree
from astropy.stats import sigma_clip
import pandas as pd
from multiprocessing import Pool
import time
import math
from matplotlib.ticker import MaxNLocator
from matplotlib.backends.backend_pdf import PdfPages
import astropy.time
from astropy.stats import sigma_clip
%matplotlib inline

In [2]:
import jdaviz

In [3]:
from jdaviz import SpecViz
from jdaviz import CubeViz
from jdaviz import MosViz
from jdaviz import Imviz

In [4]:
import warnings
warnings.simplefilter('ignore')

NIRSpec MOS\
NIRSpec IFU\
NIRSpec FS\
NIRSpec BOTS\
\
MIRI Imager\
MIRI LRS\
MIRI MRS\
MIRI Coronagraphic Imaging\
\
NIRCam Imaging\
NIRCam Coronagraphic Imaging\
NIRCam Grism TSO\
NIRCam WFSS\
\
NIRISS Imaging\
NIRISS WFSS\
NIRISS SOSS\
NIRISS AMI\

## NIRSpec MOS

In [5]:
mosviz = MosViz()
mosviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

IndexError: list index out of range

IndexError: list index out of range

The MOSViz parsers accepts lists of `Spectrum1D`, `SpectralCube`, and `CCDData` for 1D, 2D, and image data respectively. Alternatively, and the method used below, users can provide lists of file paths and MOSViz will internally attempt to parse them as their respective data type.

In [6]:
import glob
from astropy.utils.data import download_file
import tempfile
import pathlib
from zipfile import ZipFile

# Lists of data products
slit_id = []
spectra_1d = []
spectra_2d = []
cutouts = []

#Central Store data directories
file_location = "central"
if file_location == "central":
    cutouts_path = "/user/jotor/mosviz-cutouts/"
    level3_path = "/grp/jwst/wit4/nirspec/Muzerolle/Sim_products/CEERS_PRISM/"
else:
    #Local data directories
    level3_path = "/Users/pogle/Desktop/newcomputer/CEERS/CEERS_PRISM/"
    cutouts_path = "/Users/pogle/Desktop/newcomputer/CEERS/CEERS_PRISM_cutouts/"

for file_path in glob.glob(str(level3_path+'*')):
    if ('x1d' in file_path) & ('combined' in file_path):
        spectra_1d.append(file_path)
        if file_location == "central":
            slitid = file_path[88:93]
        else:
            slitid = file_path[81:86]
        slit_id.append(slitid)
    elif ('s2d' in file_path) & ('combined' in file_path):
        spectra_2d.append(file_path)
#for file_path in glob.glob(str(cutouts_path+'*')):
#    if ('cutout' in file_path) & ('.fits' in file_path):
#        cutouts.append(file_path)   
 
#Sort the slit ids and filenames
slit_id_sorted = sorted (slit_id)    
spectra_1d_sorted = sorted(spectra_1d)
spectra_2d_sorted = sorted(spectra_2d)

#Cutout filenames
cutouts_sorted = []
for s_id in slit_id_sorted: cutouts_sorted.append(cutouts_path+"cutout"+s_id+".fits")
    
#Print out the matched filenames for each slit
for s_id, f_1d, f_2d, f_cut in zip(slit_id_sorted, spectra_1d_sorted, spectra_2d_sorted, cutouts_sorted): 
    if file_location == "central":
        print(s_id, "..."+f_1d[87:], "..."+f_2d[87:], "..."+f_cut[27:])
    else:
        print(s_id, "..."+f_1d[80:], "..."+f_2d[80:], "..."+f_cut[59:])    



01507 ...s01507_combined_noadjustment_x1d.fits ...s01507_combined_noadjustment_s2d.fits ...cutout01507.fits
02140 ...s02140_combined_noadjustment_x1d.fits ...s02140_combined_noadjustment_s2d.fits ...cutout02140.fits
03339 ...s03339_combined_noadjustment_x1d.fits ...s03339_combined_noadjustment_s2d.fits ...cutout03339.fits
05136 ...s05136_combined_noadjustment_x1d.fits ...s05136_combined_noadjustment_s2d.fits ...cutout05136.fits
05803 ...s05803_combined_noadjustment_x1d.fits ...s05803_combined_noadjustment_s2d.fits ...cutout05803.fits
05947 ...s05947_combined_noadjustment_x1d.fits ...s05947_combined_noadjustment_s2d.fits ...cutout05947.fits
06811 ...s06811_combined_noadjustment_x1d.fits ...s06811_combined_noadjustment_s2d.fits ...cutout06811.fits
07572 ...s07572_combined_noadjustment_x1d.fits ...s07572_combined_noadjustment_s2d.fits ...cutout07572.fits
07640 ...s07640_combined_noadjustment_x1d.fits ...s07640_combined_noadjustment_s2d.fits ...cutout07640.fits
11836 ...s11836_combined_noa

In [7]:
#mosviz.load_data(spectra_1d, spectra_2d, images=images)

# Or alternatively: 
mosviz.load_metadata(cutouts_sorted[0:15])           #Only show the first 15, otherwise it takes too long (8min)  
mosviz.load_1d_spectra(spectra_1d_sorted[0:15])  
mosviz.load_2d_spectra(spectra_2d_sorted[0:15]) 
mosviz.load_images(cutouts_sorted[0:15])

2021-06-25 13:59:51,481 - stpipe - WARNING - No 'BUNIT' defined in the header, using 'Jy'.
2021-06-25 13:59:51,665 - stpipe - WARNING - No 'BUNIT' defined in the header, using 'Jy'.
2021-06-25 13:59:51,854 - stpipe - WARNING - No 'BUNIT' defined in the header, using 'Jy'.
2021-06-25 13:59:52,045 - stpipe - WARNING - No 'BUNIT' defined in the header, using 'Jy'.
2021-06-25 13:59:52,243 - stpipe - WARNING - No 'BUNIT' defined in the header, using 'Jy'.
2021-06-25 13:59:52,449 - stpipe - WARNING - No 'BUNIT' defined in the header, using 'Jy'.
2021-06-25 13:59:52,633 - stpipe - WARNING - No 'BUNIT' defined in the header, using 'Jy'.
2021-06-25 13:59:52,821 - stpipe - WARNING - No 'BUNIT' defined in the header, using 'Jy'.
2021-06-25 13:59:53,004 - stpipe - WARNING - No 'BUNIT' defined in the header, using 'Jy'.
2021-06-25 13:59:53,202 - stpipe - WARNING - No 'BUNIT' defined in the header, using 'Jy'.
2021-06-25 13:59:53,411 - stpipe - WARNING - No 'BUNIT' defined in the header, using 'Jy'.

## NIRSpec IFU

In [8]:
#level3
cube = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/NIRSpec/IFU/q3d_pipeline_products_20210507/NRS00001-QG-F100LP-G140H_comb_1234_g140h-f100lp_s3d.fits')

In [9]:
cubeviz = CubeViz()
cubeviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [10]:
cubeviz.load_data(cube)

2021-06-25 14:12:01,547 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-25 14:12:01,578 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-25 14:12:01,584 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-25 14:12:02,229 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-25 14:12:02,500 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-25 14:12:02,759 - stpipe - WARNING - No spectral axis found; header may be non-compliant.


In [11]:
from jdaviz import SpecViz
specviz = SpecViz()

In [12]:
#Developer note: How do you scroll interactingvely with the mouse?
specviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [13]:
#level3
spectrum = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/NIRSpec/IFU/q3d_pipeline_products_20210507/NRS00001-QG-F100LP-G140H_comb_1234_g140h-f100lp_x1d.fits')
specviz.load_spectrum(spectrum, "IFU 1D")

In [14]:
#level2
cube = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/NIRSpec/IFU/q3d_pipeline_products_20210507/NRS00001-QG-F100LP-G140H-03_1_491_SE_2020-04-23T12h15m0_noflat_s3d.fits')

In [15]:
cubeviz = CubeViz()
cubeviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [16]:
cubeviz.load_data(cube)

2021-06-25 14:14:35,175 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-25 14:14:35,194 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-25 14:14:35,199 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-25 14:14:35,595 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-25 14:14:35,798 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-25 14:14:36,015 - stpipe - WARNING - No spectral axis found; header may be non-compliant.


In [17]:
#level2
specviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [18]:
spectrum = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/NIRSpec/IFU/q3d_pipeline_products_20210507/NRS00001-QG-F100LP-G140H-04_1_492_SE_2020-04-23T12h15m0_noflat_x1d.fits')
specviz.load_spectrum(spectrum, "IFU 1D")

2021-06-25 14:15:30,247 - stpipe - WARNING - No data item with id '88e35d5f-77b3-4a07-a180-8a627c1957fb' found in viewer '42ca0252-abac-4e39-bcf8-51e4e35b9cc4'.


## NIRSpec FS

In [19]:
from jdaviz import SpecViz
specviz = SpecViz()

In [20]:
#Developer note: How do you scroll interactingvely with the mouse?
specviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [22]:
#level2
spectrum = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/NIRSpec/FS/jwebbinar/nirspec_data/nirspec_fssim_d1_x1d.fits')
specviz.load_spectrum(spectrum, "FS 1D")

In [23]:
from jdaviz import Specviz2d
specviz2d = Specviz2d()

In [24]:
#Developer note: How do you scroll interactingvely with the mouse?
specviz2d.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [25]:
spectrum = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/NIRSpec/FS/jwebbinar/nirspec_data/nirspec_fssim_d1_s2d.fits')
specviz2d.load_data(spectrum)

KeyError: 'wcs'

In [26]:
#Developer note: How do you scroll interactingvely with the mouse?
specviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [28]:
#level3
spectrum = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/NIRSpec/FS/jwebbinar/nirspec_data/nirspec_fssim_s00001_nirspec_clear-prism-s200a1-subs200a1_x1d.fits')
specviz.load_spectrum(spectrum, "FS 1D")

2021-06-25 14:28:39,305 - stpipe - WARNING - No data item with id '8b04d39c-818e-49a6-8756-f14ac9e0bdd7' found in viewer '7f38e055-a7af-4ad6-ade6-e0e5a5b476a8'.


In [29]:
#Developer note: How do you scroll interactingvely with the mouse?
specviz2d.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [30]:
#level3
spectrum = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/NIRSpec/FS/jwebbinar/nirspec_data/nirspec_fssim_s00001_nirspec_clear-prism-s200a1-subs200a1_s2d.fits')
specviz2d.load_data(spectrum)

## NIRSpec BOTS


In [42]:
#Not sure we will support?  But some possible code below

In [31]:
#import pickle
#fn_jw='/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/NIRSpec/BOTS/jwst_data.pickle'
#dbfile = open(fn_jw, 'rb') # for reading also binary mode is important
#jwst_data = pickle.load(dbfile)
#print('Loading JWST data from Pickle File')
#bjd            =jwst_data['bjd']
#wsdata_all     =jwst_data['wsdata_all']
#shx            =jwst_data['shx']
#shy            =jwst_data['shy']
#common_mode    =jwst_data['common_mode']
#all_spec       =jwst_data['all_spec']
#exposure_length=jwst_data['exposure_length']
#dbfile.close()    

In [32]:
##Sort data
#srt=np.argsort(bjd) #index to sort
#bjd=bjd[srt]
##total_counts=total_counts[srt]
#exposure_length=exposure_length[srt]
#all_spec[:,:,:]=all_spec[:,:,srt]
#
## Get Wavelegnth of Data
##file_wave = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/transit_spectroscopy_notebook/JWST_NIRSpec_wavelength_microns.txt')
#file_wave = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/NIRSpec/BOTS/JWST_NIRSpec_wavelength_microns.txt'
#f = open(file_wave, 'r')
#wsdata_all = np.genfromtxt(f)
#    
#print('wsdata size :',wsdata_all.shape)
#print('Data wavelength Loaded :',wsdata_all)
#print('wsdata new size :',wsdata_all.shape)

In [33]:
##---------------------------------------------------------
## Read in Detrending parameters
## Mean of parameter must be 0.0 to be properly normalized
## Idealy standard deviation of paramter = 1.0
##file_xy=download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/transit_spectroscopy_notebook/JWST_NIRSpec_Xposs_Yposs_CM_detrending.txt')
#file_xy='/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/NIRSpec/BOTS/JWST_NIRSpec_Xposs_Yposs_CM_detrending.txt'
#f = open(file_xy, 'r')
#data = np.genfromtxt(f, delimiter=',')
#shx        = data[:,0]
#shy        = data[:,1]
#common_mode= data[:,2]
#    
##Store Data into a pickle file
#jwst_data={'bjd':bjd, 'wsdata_all':wsdata_all, 'shx':shx , 'shy':shy , 'common_mode':common_mode, 'all_spec':all_spec, 'exposure_length':exposure_length}
#dbfile = open('jwst_data.pickle', 'ab') # Its important to use binary mode
#pickle.dump(jwst_data,dbfile)
#dbfile.close()
##---------------------------------------------------------

In [34]:
##2D Spectra
#from jdaviz import Specviz2d
#specviz2d = Specviz2d()

In [35]:
##Developer note: How do you scroll interactingvely with the mouse?
#specviz2d.app

In [36]:
##spectrum = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/NIRSpec/FS/jw00636-o019_s00002_nirspec_f100lp-g140m-s200a2-subs200a2_s2d.fits'
#expnum=2 
#specviz2d.load_data(all_spec[:,:,expnum])

In [37]:
#import matplotlib.ticker as ticker
#
#expnum=2                                           #Choose Exposure number to view
#
#plt.rcParams['figure.figsize'] = [10.0, 3.0]           # Figure dimensions
#plt.rcParams['figure.dpi']     = 200                   # Resolution
#plt.rcParams['savefig.dpi']    = 200
#plt.rcParams['image.aspect']   = 5                     # Aspect ratio (the CCD is quite long!!!)
#plt.cmap = plt.cm.magma
#plt.cmap.set_bad('k',1.)
#plt.rcParams['image.cmap'] = 'magma'                   # Colormap.
#plt.rcParams['image.interpolation'] = None
#plt.rcParams['image.origin'] = 'lower'
#plt.rcParams['font.family'] = "monospace"
#plt.rcParams['font.monospace'] = 'DejaVu Sans Mono'
#
#img=all_spec[:,:,expnum]
#zeros=np.where(img <= 0)     #Plot on a log scale, so set zero or negitive values to a small number 
#img[zeros]=1E-10
#fig,axs = plt.subplots()
#f=axs.imshow(np.log10(img),vmin=0) #Plot image
#plt.xlabel('x-pixel')
#plt.ylabel('y-pixel')
#axs.yaxis.set_major_locator(ticker.MultipleLocator(5))
#axs.yaxis.set_minor_locator(ticker.MultipleLocator(1))
#axs.xaxis.set_major_locator(ticker.MultipleLocator(50))
#axs.xaxis.set_minor_locator(ticker.MultipleLocator(10))
#plt.title('2D NIRSpec Image of Exposure '+str(expnum))
#fig.colorbar(f,label='Log$_{10}$ Electron counts',ax=axs)
#plt.show()

In [38]:
#all_spec.shape
#y_lower = 11                                             # Lower extraction aperature
#y_upper = 18                                             # Upper extraction aperature
#all_spec_1D=np.sum(all_spec[y_lower:y_upper,:,:],axis=0) # Sum along Y-axis from pixels 11 to 18
#
##Plot 
#
#plt.rcParams['figure.figsize'] = [10.0, 3.0]           # Figure dimensions
#plt.rcParams['figure.dpi']     = 200                   # Resolution
#plt.rcParams['savefig.dpi']    = 200
#plt.rcParams['image.aspect']   = 5                     # Aspect ratio (the CCD is quite long!!!)
#plt.cmap = plt.cm.magma
#plt.cmap.set_bad('k',1.)
#plt.rcParams['image.cmap'] = 'magma'                   # Colormap.
#plt.rcParams['image.interpolation'] = None
#plt.rcParams['image.origin'] = 'lower'
#plt.rcParams['font.family'] = "monospace"
#plt.rcParams['font.monospace'] = 'DejaVu Sans Mono'
#
#img=all_spec[:,:,expnum]
#zeros=np.where(img <= 0)     #Plot on a log scale, so set zero or negitive values to a small number 
#img[zeros]=1E-10
#fig,axs = plt.subplots()
#f=axs.imshow(np.log10(img),vmin=0) #Plot image
#plt.xlabel('x-pixel')
#plt.ylabel('y-pixel')
#axs.yaxis.set_major_locator(ticker.MultipleLocator(5))
#axs.yaxis.set_minor_locator(ticker.MultipleLocator(1))
#axs.xaxis.set_major_locator(ticker.MultipleLocator(50))
#axs.xaxis.set_minor_locator(ticker.MultipleLocator(10))
#plt.axhline(y_lower, color = 'w', ls = 'dashed')
#plt.axhline(y_upper, color = 'w', ls = 'dashed')
#plt.title('2D NIRSpec Image of Exposure '+str(expnum))
#fig.colorbar(f,label='Log$_{10}$ Electron counts',ax=axs)
#plt.show()

In [39]:
#fig,axs = plt.subplots()
#f=plt.plot(wsdata_all,all_spec_1D[:,0], linewidth=2,zorder=0)  #overplot Transit model at data
#plt.xlabel('Wavelength ($\mu$m)')
#plt.ylabel('Flux (e-)')
#axs.xaxis.set_major_locator(ticker.MultipleLocator(0.5))
#axs.xaxis.set_minor_locator(ticker.MultipleLocator(0.1))
#plt.annotate('H$_2$O',xy=(3.0,42000))
#plt.annotate('CO$_2$',xy=(4.2,42000))
#plt.show()

## NIRISS Imaging


In [ ]:
#Do not currently exist

## NIRISS WFSS

In [ ]:
#Not sure which data files to support with this

## NIRISS SOSS

In [40]:
#Not sure we will support?

## NIRISS AMI

In [41]:
#Not sure we will support?

In [43]:
## The data file is now in the current working directory
#calib_oifits = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/NIRISS/AMI/niriss_ami_binary3/calib_ABDOR_NIRISS_g7s6_F480M_59670.0.0f.oifits'
## Display OIFITS file information
#fits.info(calib_oifits)

In [44]:
##Developer Notes: Ask how to remove other files from viewer before and after opening data
#imviz.load_data(calib_oifits, data_label='NIRISS AMI Imaging')
#viewer = imviz.app.get_viewer('viewer-1')
#imviz.app

In [45]:
#from astropy.utils.data import download_file
#gc_2mass_j = download_file('https://www.astropy.org/astropy-data/galactic_center/gc_2mass_j.fits', cache=True)
#gc_msx_e = download_file('https://www.astropy.org/astropy-data/galactic_center/gc_msx_e.fits', cache=True)

In [46]:
#from jdaviz import Imviz
#import matplotlib.pyplot as plt
#
#imviz = Imviz()
#imviz.load_data(gc_2mass_j, data_label='gc_2mass_j')
#imviz.load_data(gc_msx_e, data_label='gc_msx_e', show_in_viewer=False)
#
#viewer = imviz.app.get_viewer('viewer-1')
#
#imviz.app

## NIRCam Imaging

In [48]:
nircam_imaging = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/NIRCam/Imaging/images_level3/LMC_F070W_nrcb1_combined_PrimaryDither_1_SubpxDither_4_i2d.fits')

In [49]:
from jwst import datamodels
#from jdaviz import Imviz
imviz = Imviz()

In [50]:
#Developer Notes: Ask how to remove other files from viewer before and after opening data
imviz.load_data(nircam_imaging, data_label='NIRCam Imaging')

viewer = imviz.app.get_viewer('viewer-1')

imviz.app

  warnings.warn(message="DrizProduct is deprecated and will be removed.  "

2021-06-25 16:00:23,416 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/jwst/datamodels/drizproduct.py:10: DeprecationWarning: DrizProduct is deprecated and will be removed.  Use ImageModel.
  warnings.warn(message="DrizProduct is deprecated and will be removed.  "

  warnings.warn(message="DrizProduct is deprecated and will be removed.  "

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

2021-06-25 16:00:24,177 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/glue/core/component.py:96: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy sc

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

## NIRCam Coronagraphic Imaging

In [52]:
#Not sure which data files to support with this

## NIRCam Grism TSO

In [53]:
#Not sure we will support?

## NIRCam WFSS

In [54]:
#Not sure which data files to support with this

## MIRI Imaging

In [59]:
miri_imaging = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/MIRI/Imaging/mirisim/complex_scene_F770W_combined_i2d.fits')

  and should_run_async(code)

2021-06-25 16:11:57,189 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

  and should_run_async(code)



In [60]:
from jwst import datamodels
imviz = Imviz()

object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)

2021-06-25 16:12:04,458 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Figure).__init__(scales={'x': LinearScale(allow_padding=False, max=1.0, min=0.0), 'y': LinearScale(max=1.0, min=0.0)}).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)

object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)



In [61]:
#Developer Notes: Ask how to remove other files from viewer before and after opening data
imviz.load_data(nircam_imaging, data_label='NIRCam Imaging')

viewer = imviz.app.get_viewer('viewer-1')

imviz.app

  warnings.warn(message="DrizProduct is deprecated and will be removed.  "

2021-06-25 16:12:08,227 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/jwst/datamodels/drizproduct.py:10: DeprecationWarning: DrizProduct is deprecated and will be removed.  Use ImageModel.
  warnings.warn(message="DrizProduct is deprecated and will be removed.  "

  warnings.warn(message="DrizProduct is deprecated and will be removed.  "

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

2021-06-25 16:12:08,641 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/glue/core/component.py:96: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy sc

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

## MIRI LRS Slit

In [63]:
from jdaviz import Specviz2d
specviz2d = Specviz2d()

object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)

2021-06-25 16:14:07,156 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Figure).__init__(scales={'x': LinearScale(allow_padding=False, max=1.0, min=0.0), 'y': LinearScale(max=1.0, min=0.0)}).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)

object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)



In [64]:
#Developer note: How do you scroll interactingvely with the mouse?
specviz2d.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [65]:
#Level3
spectrum = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/MIRI/LRS/det_image_seq5_MIRIMAGE_P750Lexp1_s2d.fits')
specviz2d.load_data(spectrum)

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

2021-06-25 16:14:14,924 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/glue/core/component.py:96: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

Deprecated in NumPy 1.20; for more details and

KeyError: "Keyword 'WAVEND' not found."

In [69]:
from jdaviz import SpecViz
specviz = SpecViz()

In [70]:
#Developer note: How do you scroll interactingvely with the mouse?
specviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [71]:
#Level3
spectrum = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/MIRI/LRS/det_image_seq5_MIRIMAGE_P750Lexp1_x1d.fits')
specviz.load_spectrum(spectrum, "LRS 1D")

RuntimeError: Keyword SRCTYPE is None.  It should be 'POINT' or 'EXTENDED'. Can't decide between `flux` and `surf_bright` columns.

## MIRI MRS

In [83]:
#Level 3 All Cube
cube = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/MIRI/MRS/mirisim/allcube_ch1-2-3-4-shortlongmedium-_s3d.fits')

  and should_run_async(code)

2021-06-25 16:29:56,204 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

  and should_run_async(code)



In [84]:
cubeviz = CubeViz()
cubeviz.app

object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)

2021-06-25 16:36:19,758 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Figure).__init__(scales={'x': LinearScale(allow_padding=False, max=1.0, min=0.0), 'y': LinearScale(max=1.0, min=0.0)}).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)

object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)



Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [85]:
cubeviz.load_data(cube)

2021-06-25 16:36:28,831 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-25 16:36:30,149 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-25 16:36:30,156 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
  logging.warn(e)

2021-06-25 16:36:30,291 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/jdaviz/configs/cubeviz/plugins/parsers.py:105: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn(e)

  logging.warn(e)

2021-06-25 16:36:30,293 - stpipe - WARNING - 'NoneType' object has no attribute 'to_header'
2021-06-25 16:36:30,296 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
  logging.warn(e)

2021-06-25 16:36:30,299 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/jdaviz/configs/cubeviz/plugins/parsers.py:105: DeprecationWarning: The 'warn' function is deprecated

In [76]:
from jdaviz import SpecViz
specviz = SpecViz()

In [77]:
#Level3 All Cube 1D
#Developer note: How do you scroll interactingvely with the mouse?
specviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [78]:
#spectrum = '/Users/ofox/data/jwst/dat_pyinthesky/jdat_notebooks/helper_classes/data/MIRI/MRS/combine_dithers_all_exposures_ch1-medium_x1d.fits'
spectrum = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/MIRI/MRS/mirisim/allcube_ch1-2-3-4-shortlongmedium-_x1d.fits')


specviz.load_spectrum(spectrum, "MRS 1D")

In [81]:
#Level 3 Chan Cube
cube = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/MIRI/MRS/mirisim/chancube_ch1-longshortmedium-_s3d.fits')

In [79]:
cubeviz = CubeViz()
cubeviz.app

  and should_run_async(code)

2021-06-25 16:27:03,409 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

  and should_run_async(code)

object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)

2021-06-25 16:27:03,545 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Figure).__init__(scales={'x': LinearScale(allow_padding=False, max=1.0, min=0.0), 'y': LinearScale(max

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [82]:
cubeviz.load_data(cube)

2021-06-25 16:29:00,049 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-25 16:29:00,080 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-25 16:29:00,085 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dt = np.find_common_type([data.dtype], [np.float])

2021-06-25 16:29:00,238 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/spectral_cube/masks.py:228: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dt = np.find_commo

2021-06-25 16:29:00,418 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/glue/core/component.py:96: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

2021-06-25 16:29:00,422 - stpipe - WARNING - /

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

2021-06-25 16:29:00,972 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/glue/core/component.py:96: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

Deprecated in NumPy 1.20; for more details and

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if hdu.data.dtype in (np.int, np.uint, np.uint32) or \

2021-06-25 16:29:01,227 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/jdaviz/configs/cubeviz/plugins/parsers.py:114: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if hdu.data.dtype in (np.int, np.uint, np.uint32) or \

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprec

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dt = np.find_common_type([data.dtype], [np.float])

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif isinstance(c1[1], (float, np.float)):

2021-06-25 16:29:01,367 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/spectral_cube/wcs_utils.py:395: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif isinstance(c1[1], (float, np.float)):

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.

In [86]:
#Level3 Chan Cube 1D
#Developer note: How do you scroll interactingvely with the mouse?
specviz.app

  and should_run_async(code)

2021-06-25 16:37:06,685 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

  and should_run_async(code)



Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [87]:
spectrum = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/MIRI/MRS/mirisim/chancube_ch1-longshortmedium-_x1d.fits')
specviz.load_spectrum(spectrum, "MRS 1D")

2021-06-25 16:37:14,126 - stpipe - WARNING - No data item with id 'c0868863-48ba-421c-9c77-6946238ff05c' found in viewer 'e944bc1a-9644-4c4a-ad1e-3d501c987e28'.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

2021-06-25 16:37:14,129 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/glue/core/component.py:96: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.

In [88]:
#Level 3 Single Cube
cube = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/MIRI/MRS/mirisim/Level3_ch1-long_s3d.fits')

  and should_run_async(code)

2021-06-25 16:37:17,998 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

  and should_run_async(code)



In [89]:
cubeviz = CubeViz()
cubeviz.app

object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)

2021-06-25 16:37:32,119 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Figure).__init__(scales={'x': LinearScale(allow_padding=False, max=1.0, min=0.0), 'y': LinearScale(max=1.0, min=0.0)}).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)

object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)



Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [90]:
cubeviz.load_data(cube)

2021-06-25 16:37:37,532 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-25 16:37:37,546 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-25 16:37:37,552 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dt = np.find_common_type([data.dtype], [np.float])

2021-06-25 16:37:37,701 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/spectral_cube/masks.py:228: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dt = np.find_commo

2021-06-25 16:37:37,776 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/glue/core/component.py:96: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

2021-06-25 16:37:37,779 - stpipe - WARNING - /

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

2021-06-25 16:37:38,128 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/glue/core/component.py:96: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

Deprecated in NumPy 1.20; for more details and

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if hdu.data.dtype in (np.int, np.uint, np.uint32) or \

2021-06-25 16:37:38,327 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/jdaviz/configs/cubeviz/plugins/parsers.py:114: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if hdu.data.dtype in (np.int, np.uint, np.uint32) or \

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprec

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dt = np.find_common_type([data.dtype], [np.float])

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif isinstance(c1[1], (float, np.float)):

2021-06-25 16:37:38,451 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/spectral_cube/wcs_utils.py:395: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif isinstance(c1[1], (float, np.float)):

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.

In [91]:
#Level3 Single Cube 1D
specviz.app

  and should_run_async(code)

2021-06-25 16:37:46,017 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

  and should_run_async(code)



Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [92]:
spectrum = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/MIRI/MRS/mirisim/Level3_ch1-long_x1d.fits')
specviz.load_spectrum(spectrum, "MRS 1D")

2021-06-25 16:37:51,331 - stpipe - WARNING - No data item with id 'c0868863-48ba-421c-9c77-6946238ff05c' found in viewer 'e944bc1a-9644-4c4a-ad1e-3d501c987e28'.
2021-06-25 16:37:51,334 - stpipe - WARNING - No data item with id '9a91724a-cd61-40ca-bce9-a49680e3d982' found in viewer 'e944bc1a-9644-4c4a-ad1e-3d501c987e28'.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

2021-06-25 16:37:51,336 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/glue/core/component.py:96: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.

In [93]:
#Level 2 Single Cube
cube = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/MIRI/MRS/mirisim/Level2_det_image_seq1_MIRIFULONG_34LONGexp1_s3d.fits')

  and should_run_async(code)

2021-06-25 16:37:59,885 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

  and should_run_async(code)



In [94]:
cubeviz = CubeViz()
cubeviz.app

object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)

2021-06-25 16:38:12,917 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/ipywidgets/widgets/widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Figure).__init__(scales={'x': LinearScale(allow_padding=False, max=1.0, min=0.0), 'y': LinearScale(max=1.0, min=0.0)}).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)

object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)



Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [95]:
cubeviz.load_data(cube)

2021-06-25 16:38:17,172 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-25 16:38:17,188 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
2021-06-25 16:38:17,195 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
  logging.warn(e)

2021-06-25 16:38:17,204 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/jdaviz/configs/cubeviz/plugins/parsers.py:105: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn(e)

  logging.warn(e)

2021-06-25 16:38:17,205 - stpipe - WARNING - 'NoneType' object has no attribute 'to_header'
2021-06-25 16:38:17,208 - stpipe - WARNING - No spectral axis found; header may be non-compliant.
  logging.warn(e)

2021-06-25 16:38:17,211 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/jdaviz/configs/cubeviz/plugins/parsers.py:105: DeprecationWarning: The 'warn' function is deprecated

In [96]:
#Level2 Single Cube 1D
specviz.app

  and should_run_async(code)

2021-06-25 16:38:35,947 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

  and should_run_async(code)



Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [97]:
spectrum = download_file('https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/helper_classes/MIRI/MRS/mirisim/Level2_det_image_seq1_MIRIFULONG_34LONGexp1_x1d.fits')
specviz.load_spectrum(spectrum, "MRS 1D")

2021-06-25 16:38:43,061 - stpipe - WARNING - No data item with id 'c0868863-48ba-421c-9c77-6946238ff05c' found in viewer 'e944bc1a-9644-4c4a-ad1e-3d501c987e28'.
2021-06-25 16:38:43,064 - stpipe - WARNING - No data item with id '9a91724a-cd61-40ca-bce9-a49680e3d982' found in viewer 'e944bc1a-9644-4c4a-ad1e-3d501c987e28'.
2021-06-25 16:38:43,066 - stpipe - WARNING - No data item with id 'b9ca3886-d226-480e-b28d-904dff741407' found in viewer 'e944bc1a-9644-4c4a-ad1e-3d501c987e28'.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.can_cast(self[(0,) * self.ndim].dtype, np.complex)

2021-06-25 16:38:43,068 - stpipe - WARNING - /Users/ofox/miniconda3/envs/helper/lib/python3.9/site-packages/glue/core/component.py:96: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you spec

## MIRI Coronagraphic Imaging

In [72]:
#Not sure which data files to support with this